In [1]:
import ftplib 
from ftplib import FTP
import subprocess
import pip._internal.commands as commands
from rpy2.robjects.packages import importr

ModuleNotFoundError: No module named 'rpy2'

In [11]:
HOSTNAME = 'ftp.datasus.gov.br'
ftp_server = ftplib.FTP(HOSTNAME)
ftp_server.login() 
ftp_server.cwd('dissemin/publicos/SINAN/DADOS/FINAIS/')
filename = "LEIVBR07.dbc"
with open(filename, "wb") as file:    
    ftp_server.retrbinary(f"RETR {filename}", file.write) 

In [30]:
def dbccsv(raw_filename):
    dbccsv_path = "dbc_csv.R" + " " + raw_filename

    try:
        r_script_path = commands.getstatusoutput('which Rscript')[1]
        subprocess.call(r_script_path + " --vanilla " + dbccsv_path, shell=True)
        return True
    except:
        print("(Rscript) Error converting file: " + raw_filename)

    return False

In [31]:
raw_filename="LEIVBR07.dbc"
teste=dbccsv(raw_filename)

(Rscript) Error converting file: LEIVBR07.dbc
